In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Load raw data
df = pd.read_csv('official_row_data.csv', encoding='utf-8')

# Pivot to wide format
df_cross = (
    df
    .set_index(['story', 'model', 'session'])['score']
    .unstack(['model', 'session'])
    .reset_index()
)

# Rename columns to 'model-session'
df_cross.columns = [
    col if isinstance(col, str) else f"{col[0]}-{col[1]}"
    for col in df_cross.columns
]

# Extract numeric matrix for PCA
X = df_cross.drop('story-', axis=1).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA()
loadings = pca.fit(X_scaled).components_.T * np.sqrt(pca.explained_variance_)

# Format loadings
max_pcs = min(4, X.shape[1])
loadings_df = pd.DataFrame(
    loadings[:, :max_pcs],
    index=df_cross.columns[1:],
    columns=[f'PC{i+1}' for i in range(max_pcs)]
).reset_index().rename(columns={'index': 'session_id'})

# Clustering (K=5)
X_pca = loadings_df[[f'PC{i+1}' for i in range(max_pcs)]].values
kmeans = KMeans(n_clusters=5, random_state=42)
loadings_df['cluster'] = kmeans.fit_predict(X_pca)

# Merge cluster labels back
df['session_id'] = df['model'] + '-' + df['session'].astype(str)
df = pd.merge(df, loadings_df[['session_id', 'cluster']], on='session_id', how='left')

# Remove specific sessions
drop_list = [f'Notebook LM-{i}' for i in range(2, 8)]
df_filtered = df[~df['session_id'].isin(drop_list)]

# Group comments by cluster
comments = {
    cluster: df_filtered[df_filtered['cluster'] == cluster]['comment'].dropna().tolist()
    for cluster in sorted(df_filtered['cluster'].unique())
}

# Text preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

custom_stopwords = [
    'the', 'a', 'an', 'in', 'on', 'of', 'to', 'with', 'at', 'by', 'for', 'from',
    'is', 'are', 'was', 'were', 'be', 'being', 'been',
    'that', 'this', 'these', 'those', 'and', 'but', 'or', 'if', 'then', 'so',
    'as', 'because', 'while', 'although', 'though', 'yet', 'also', 'just', 'it',
    'its', 'fs', 'about', 'ai', 'narrative', 'story', 'through', 'slightly',
    'somewhat', 'effectively', 'well', 'occasionally'
]

# TF-IDF per cluster
ranking = []
for cluster_id, texts in comments.items():
    processed = [preprocess_text(t) for t in texts]
    vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
    tfidf_matrix = vectorizer.fit_transform(processed)

    feature_names = vectorizer.get_feature_names_out()
    mean_scores = tfidf_matrix.mean(axis=0).A1
    tfidf_scores = dict(zip(feature_names, mean_scores))

    top_words = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for rank, (word, _) in enumerate(top_words, 1):
        ranking.append([cluster_id, rank, word])

# Create pivot table
df_keywords = pd.DataFrame(ranking, columns=['cluster', 'rank', 'word'])
pivot_df = df_keywords.pivot(index='rank', columns='cluster', values='word')
pivot_df.columns = [f'Cluster {i}' for i in pivot_df.columns]

# Display
print("\nTable E-2: Top 10 TF-IDF Words per Cluster")
print(pivot_df)

print("\nTable E-2: Number of Comments per Cluster")
for cluster_id, comment_list in comments.items():
    print(f"Cluster {cluster_id}: {len(comment_list)} comments")
